In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import plasma.functional as F

from plasma.meta.object_graph import Manager

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class HUHU(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [4]:
class HEHE(F.AutoPipe):

    def __init__(self, huhu, bee) -> None:
        super().__init__()
        self.huhu = huhu

In [5]:
class HAHA(F.AutoPipe):

    def __init__(self, hehe:HEHE, huhu:HUHU, test_factory:dict) -> None:
        super().__init__()
        print(test_factory)
        self.hehe = hehe
        self.huhu = huhu

In [6]:
injector = Manager()
injector2 = Manager()

In [7]:
dep_factory = injector2.init_factory('test_factory')
dep_factory

DependencyFactory(
	name=test_factory,
	egistered_names=[]
)

In [8]:
@dep_factory.register('bee')
class Bee:
    
    def __init__(self, wings=2, need=1):
        pass

@dep_factory.register('bop')
class Bop:
    
    def __init__(self, unkown):
        pass

In [9]:
injector\
    .add_dependency('huhu', HUHU)\
    .add_dependency('hehe', HEHE)\
    .add_dependency('hihi', HEHE)

temp = injector2\
    .add_dependency('haha', HAHA)\
    .merge(injector)

temp

/u02/thanh/workplace/plasma/plasma/meta/object_graph/dependency_manager.py:46: UserWarning: name collision after merging at: {'hehe', 'bee', 'huhu'}
  warn(f'name collision after merging at: {collisions}')


|->haha
  |->hehe: HEHE
    |->huhu: HUHU
      |->name
      |->age
      |->test = int
    |->bee
      |->wings = int
      |->need = int
  |->huhu...
  |->test_factory: DependencyFactory
    |->bee...
    |->bop
      |->unkown
----------------------------------------------------------------------------------------------------
|->hihi
  |->huhu...
  |->bee...
----------------------------------------------------------------------------------------------------

In [ ]:
temp.entries

In [ ]:
obj, = injector2.run('haha', huhu=8, unkown=8, name='huhu')
obj